In [1]:
import urllib.request
import time
import  bs4 as bs
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)\

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


Getting postal codes from Wiki

In [2]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
table_rows = []
table = soup.find('table')
table_rows = table.find_all('tr')

Creating a postal code dataframe:

In [3]:
l = []
for tr in table_rows[1:]:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

df = pd.DataFrame(l, columns=['PostalCode', 'Borough', 'Neighborhood'])

removing new line simbols

In [4]:
df.replace(to_replace=r'(\r\n|\r|\n)', value=' ', regex=True, inplace=True)

replacing Not Assigned with NaN

In [5]:
df['Borough'].replace(to_replace=r'Not assigned', value=np.nan, regex=True, inplace=True)

removing whitespaces from start and end

In [6]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

removing NaN's

In [7]:
df.dropna(inplace=True)

In [41]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188712
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


resetting index

In [8]:
df = df.groupby('PostalCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()

In [9]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 0, df['Borough'], df['Neighborhood'])

In [10]:
df.shape

(103, 3)

In [11]:
data_geo = pd.read_csv("/Users/anastasiakulbaka/PycharmProjects/Test/Geospatial_Coordinates.csv")
df_geo = pd.DataFrame(data=data_geo)
df_geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [12]:
cond = df['PostalCode'] == df_geo['PostalCode']
df = df.assign(Latitude = np.where(cond, df_geo['Latitude'].round(decimals=6), 'FALSE'))
df = df.assign(Longitude = np.where(cond, df_geo['Longitude'].round(decimals=6), 'FALSE'))

In [13]:
df.to_csv('df_cleaned.csv')

In [14]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188712
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
from datetime import date
today = date.today()
# ddmmYY
VERSION = today.strftime("%Y%m%d")

In [18]:
CLIENT_ID = 'I5XIAEZGJHJVXUPBQPUREOWFJ2C0KZBZXN2KK33HCHVI2QGY' # your Foursquare ID
CLIENT_SECRET = '5AWNJSIRQ1PGYICGJYC15E52DYJY2AF2MT0T1ACMZF5JOYDS' # your Foursquare Secret
#VERSION = '20200824' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Foursquare API version:' + VERSION)

radius = 500
LIMIT = 100
#url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
#url

Your credentails:
CLIENT_ID: I5XIAEZGJHJVXUPBQPUREOWFJ2C0KZBZXN2KK33HCHVI2QGY
CLIENT_SECRET:5AWNJSIRQ1PGYICGJYC15E52DYJY2AF2MT0T1ACMZF5JOYDS
Foursquare API version:20200909


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(2149, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188712,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188712,G & G Electronics,43.765309,-79.191537,Electronics Store


In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,57,57,57,57,57,57
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15


In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
toronto_onehot.shape

(2149, 266)

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [27]:
toronto_grouped.shape

(96, 266)

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2             Clothing Store  0.25
3  Latin American Restaurant  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1     Coffee Shop  0.14
2  Sandwich Place  0.14
3             Pub  0.14
4            Pool  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0              Bank  0.09
1       Coffee Shop  0.09
2  Sushi Restaurant  0.04
3       Supermarket  0.04
4       Bridal Shop  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.08
2      Sandwich Place  0.08
3    Toy / Game Store  0.

         venue  freq
0  Pizza Place  0.07
1  Coffee Shop  0.07
2         Park  0.04
3          Pub  0.04
4       Bakery  0.04


----Steeles West, L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.23
1         Grocery Store  0.15
2                  Bank  0.08
3           Pizza Place  0.08
4           Coffee Shop  0.08


----Stn A PO Boxes----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.04
2  Japanese Restaurant  0.03
3             Beer Bar  0.03
4   Seafood Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.08
1          Coffee Shop  0.08
2            Gastropub  0.05
3              Brewery  0.05
4  American Restaurant  0.05


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
                venue  freq
0         Coffee Shop  0.13
1  Light Rail Station  0.07
2        Liquor Store  0.07
3          Sports Bar  0.07
4         Supermarket  0.07


----The A

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except: 
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pub,Sandwich Place,Pool,Department Store,Curling Ice,Dance Studio,Deli / Bodega
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Pizza Place,Diner,Sandwich Place,Bridal Shop,Deli / Bodega,Restaurant,Intersection
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Grocery Store,Hobby Shop,Comfort Food Restaurant,Pharmacy,Pizza Place,Café,Butcher


In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(16) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Women's Store,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Bar,Home Service,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dance Studio
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188712,1.0,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Restaurant,Construction & Landscaping,Dessert Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Soccer Field,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Fried Chicken Joint,Caribbean Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Gas Station,Hakka Restaurant,Dog Run,Distribution Center
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1.0,Playground,Construction & Landscaping,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Coffee Shop,Bus Station,Department Store,Discount Store,Train Station,Donut Shop,Doner Restaurant,Dog Run,Drugstore,Distribution Center
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bakery,Park,Metro Station,Bus Station,Bus Line,Intersection,Ice Cream Shop,Soccer Field,Distribution Center,Discount Store
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,Motel,American Restaurant,Women's Store,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,College Stadium,Skating Rink,General Entertainment,Café,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


In [33]:
toronto_merged_n0_na = toronto_merged.dropna().reset_index()
toronto_merged_n0_na.astype({'Cluster Labels': 'int32'})

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3,Fast Food Restaurant,Women's Store,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Bar,Home Service,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dance Studio
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188712,1,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Restaurant,Construction & Landscaping,Dessert Shop
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Soccer Field,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Fried Chicken Joint,Caribbean Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Gas Station,Hakka Restaurant,Dog Run,Distribution Center
5,5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Playground,Construction & Landscaping,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
6,6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1,Coffee Shop,Bus Station,Department Store,Discount Store,Train Station,Donut Shop,Doner Restaurant,Dog Run,Drugstore,Distribution Center
7,7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1,Bakery,Park,Metro Station,Bus Station,Bus Line,Intersection,Ice Cream Shop,Soccer Field,Distribution Center,Discount Store
8,8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0,Motel,American Restaurant,Women's Store,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,College Stadium,Skating Rink,General Entertainment,Café,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


In [34]:
#type(cluster)

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_n0_na['Latitude'], toronto_merged_n0_na['Longitude'], toronto_merged_n0_na['Neighborhood'], toronto_merged_n0_na['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 0, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M1M,-79.239476,0.0,Motel,American Restaurant,Women's Store,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
13,M1T,-79.304302,0.0,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Noodle House,Bank,Italian Restaurant,Fried Chicken Joint,Thai Restaurant,Intersection,Gas Station
15,M1W,-79.318389,0.0,Fast Food Restaurant,Grocery Store,Breakfast Spot,Bank,Sandwich Place,Cosmetics Shop,Chinese Restaurant,Pizza Place,Pharmacy,Coffee Shop
23,M2R,-79.442259,0.0,Grocery Store,Bank,Pharmacy,Pizza Place,Coffee Shop,General Travel,Creperie,Doner Restaurant,Dog Run,Distribution Center
33,M4A,-79.315572,0.0,Coffee Shop,Portuguese Restaurant,Pizza Place,Hockey Arena,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
34,M4B,-79.309937,0.0,Pizza Place,Intersection,Bank,Gym / Fitness Center,Pharmacy,Breakfast Spot,Gastropub,Athletics & Sports,Women's Store,Diner
71,M6B,-79.445073,0.0,Park,Sushi Restaurant,Pizza Place,Pub,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store
88,M8W,-79.543484,0.0,Pizza Place,Coffee Shop,Gym,Pub,Sandwich Place,Pool,Department Store,Curling Ice,Dance Studio,Deli / Bodega
93,M9L,-79.565963,0.0,Pizza Place,Furniture / Home Store,Women's Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
96,M9P,-79.532242,0.0,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Intersection,Sandwich Place,Discount Store,Chinese Restaurant,Dessert Shop,Deli / Bodega,Department Store


In [37]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 1, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,-79.160497,1.0,Bar,Home Service,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dance Studio
2,M1E,-79.188712,1.0,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Restaurant,Construction & Landscaping,Dessert Shop
3,M1G,-79.216917,1.0,Coffee Shop,Korean Restaurant,Soccer Field,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
4,M1H,-79.239476,1.0,Fried Chicken Joint,Caribbean Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Gas Station,Hakka Restaurant,Dog Run,Distribution Center
5,M1J,-79.239476,1.0,Playground,Construction & Landscaping,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
6,M1K,-79.262029,1.0,Coffee Shop,Bus Station,Department Store,Discount Store,Train Station,Donut Shop,Doner Restaurant,Dog Run,Drugstore,Distribution Center
7,M1L,-79.284577,1.0,Bakery,Park,Metro Station,Bus Station,Bus Line,Intersection,Ice Cream Shop,Soccer Field,Distribution Center,Discount Store
9,M1N,-79.264848,1.0,College Stadium,Skating Rink,General Entertainment,Café,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
10,M1P,-79.273304,1.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Thrift / Vintage Store,Women's Store,Diner,Dance Studio,Deli / Bodega,Department Store
11,M1R,-79.295849,1.0,Middle Eastern Restaurant,Vietnamese Restaurant,Bakery,Sandwich Place,Auto Garage,Smoke Shop,Shopping Mall,Doner Restaurant,Dog Run,Distribution Center


In [38]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 2, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M2L,-79.374714,2.0,Cafeteria,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Curling Ice


In [39]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 3, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,3.0,Fast Food Restaurant,Women's Store,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [40]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 4, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M1V,-79.284577,4.0,Coffee Shop,Park,Playground,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Distribution Center
20,M2M,-79.408493,4.0,Park,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
22,M2P,-79.400049,4.0,Construction & Landscaping,Park,Convenience Store,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
24,M3A,-79.329656,4.0,Park,Food & Drink Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
39,M4J,-79.338106,4.0,Park,Intersection,Convenience Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
43,M4N,-79.38879,4.0,Park,Swim School,Bus Line,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Electronics Store,Diner
49,M4W,-79.377529,4.0,Park,Playground,Trail,Donut Shop,Doner Restaurant,Dog Run,Drugstore,Distribution Center,Discount Store,Cuban Restaurant
73,M6E,-79.453512,4.0,Park,Women's Store,Pool,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Cupcake Shop,Discount Store
95,M9N,-79.518188,4.0,Park,Convenience Store,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
